# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [83]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [84]:
orders = pd.read_csv(".\Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [85]:
clients_spend = orders.groupby(["CustomerID"]).agg({"amount_spent":sum})
clients_spend

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [86]:
labels = ["Basic","Preferred","VIP"]
percentiles = pd.qcut(clients_spend['amount_spent'],[0, 0.75, 0.95, 1],labels=labels)
clients_spend["customer_class"] = percentiles
clients_spend.head()

,amount_spent,customer_class
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Basic


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [87]:
clients_spend = pd.merge(left = clients_spend,
                                right = orders[["CustomerID","Country"]],
                                how = "inner",
                                left_on = "CustomerID",
                                right_on = "CustomerID")
clients_spend.head()

,CustomerID,amount_spent,customer_class,Country
0,12346,77183.6,VIP,United Kingdom
1,12347,4310.0,Preferred,Iceland
2,12347,4310.0,Preferred,Iceland
3,12347,4310.0,Preferred,Iceland
4,12347,4310.0,Preferred,Iceland


In [88]:
clients_spend = clients_spend.groupby(["customer_class","Country"]).agg({"customer_class":"count"})

In [89]:
clients_spend.rename(columns = {'customer_class':'customer_class_count'}, inplace = True)
clients_spend = clients_spend.reset_index()
clients_spend.rename(columns = {'Country':'country'}, inplace = True)

,customer_class,country,customer_class_count
0,Basic,Australia,157
1,Basic,Austria,240
2,Basic,Bahrain,17
3,Basic,Belgium,474
4,Basic,Brazil,32
...,...,...,...
106,VIP,Switzerland,594
107,VIP,USA,0
108,VIP,United Arab Emirates,0
109,VIP,United Kingdom,84185


In [100]:
condition_VIP = clients_spend["customer_class"]=="VIP"
VIP_cust_count = clients_spend[condition_VIP].sort_values(by="customer_class_count",ascending=False)
VIP_cust_count.head()

,customer_class,country,customer_class_count
109,VIP,United Kingdom,84185
84,VIP,EIRE,7077
87,VIP,France,3290
88,VIP,Germany,3127
97,VIP,Netherlands,2080


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [101]:
condition_PREF = clients_spend["customer_class"]=="Preferred"
PREF_cust_count = clients_spend[condition_PREF].sort_values(by="customer_class_count",ascending=False)
PREF_cust_count.head()

,customer_class,country,customer_class_count
72,Preferred,United Kingdom,137450
51,Preferred,Germany,4222
50,Preferred,France,3011
40,Preferred,Belgium,1503
67,Preferred,Spain,1058


In [102]:
VIP_PREF = pd.merge(left = VIP_cust_count[["country","customer_class_count"]],
                                right = PREF_cust_count[["country","customer_class_count"]],
                                how = "inner",
                                left_on = "country",
                                right_on = "country")
VIP_PREF.head()

,country,customer_class_count_x,customer_class_count_y
0,United Kingdom,84185,137450
1,EIRE,7077,161
2,France,3290,3011
3,Germany,3127,4222
4,Netherlands,2080,0


In [108]:
VIP_PREF["VIP+PREF_count"]=VIP_PREF["customer_class_count_x"]+VIP_PREF["customer_class_count_y"]
VIP_PREF[["country","VIP+PREF_count"]].sort_values(by="VIP+PREF_count",ascending=False).head()

,country,VIP+PREF_count
0,United Kingdom,221635
3,Germany,7349
1,EIRE,7238
2,France,6301
4,Netherlands,2080
